# eRO-ExTra database setup

This notebook creates a SQL database **eroextra.db** containing the full catalog of 304 eRO-ExTra sources.

### The following tables are created:

- **"eroextra_master_erosita"** - the master catalog of 304 X-ray sources with eROSITA with their key X-ray properties
- **"optical_follow_up"** - a table of optical follow-up observations conducted this study, linked to corresponding X-ray sources

In [1]:
import sqlite3
from astropy.io import fits
import ast
import numpy as np

from fits_utils import parse_numeric_field,parse_instrument_field,safe_field_access

In [16]:
#load the catalog which you would like to use to create a SQL database
# for this project we use the master ero-extra catalog

eroextra_catalog_fits = 'data/eroextra_final.fits'
with fits.open(eroextra_catalog_fits) as hdul:
    eroextra_data = hdul[1].data

In [3]:
#load any additional tables to add to the database
#load the text file with optical follow-up data
with open("data/optical_data.txt", "r") as f:
    content = f.read()
optical_data = ast.literal_eval(content)

In [4]:
#launch a DB connection
conn = sqlite3.connect("eroextra.db")
cursor = conn.cursor()

### Create tables for the database

In [6]:
# Create the eroextra_master_erosita table
cursor.execute("DROP TABLE IF EXISTS eroextra_master_erosita;")
cursor.execute("""
CREATE TABLE eroextra_master_erosita (
    ero_name TEXT PRIMARY KEY,
    ero_ra REAL,
    ero_dec REAL,
    ero_poserr REAL,
    ero_flux_erass1 REAL,
    ero_flux_erass2 REAL,
    ero_flux_erass3 REAL,
    ero_flux_erass4 REAL,
    ero_flux_erass5 REAL,
    ero_flux_err_erass1 REAL,
    ero_flux_err_erass2 REAL,
    ero_flux_err_erass3 REAL,
    ero_flux_err_erass4 REAL,
    ero_flux_err_erass5 REAL,
    amplitude REAL,
    significance REAL,
    ero_lcclass REAL,
    ero_date TEXT,
    nh REAL,
    ero_gamma REAL,
    ero_gamma_err REAL,
    ero_cstat_dof TEXT,
    ero_flux_mod REAL,
    ero_flux_mod_err REAL
);
""")

insert_sql = """
INSERT OR REPLACE INTO eroextra_master_erosita (
    ero_name, ero_ra, ero_dec, ero_poserr,
    ero_flux_erass1, ero_flux_erass2, ero_flux_erass3, ero_flux_erass4, ero_flux_erass5,
    ero_flux_err_erass1, ero_flux_err_erass2, ero_flux_err_erass3, ero_flux_err_erass4, ero_flux_err_erass5,
    amplitude, significance, ero_lcclass, ero_date, nh,
    ero_gamma, ero_gamma_err, ero_cstat_dof, ero_flux_mod, ero_flux_mod_err
) VALUES (
    ?, ?, ?, ?,
    ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?
);
"""

for row in eroextra_data:
    values = (
        row['ero_name'],
        float(row['ero_ra']),
        float(row['ero_dec']),
        float(row['ero_poserr']),
        float(row['ero_flux_erass1']),
        float(row['ero_flux_erass2']),
        float(row['ero_flux_erass3']),
        float(row['ero_flux_erass4']),
        float(row['ero_flux_erass5']),
        float(row['ero_flux_err_erass1']),
        float(row['ero_flux_err_erass2']),
        float(row['ero_flux_err_erass3']),
        float(row['ero_flux_err_erass4']),
        float(row['ero_flux_err_erass5']),
        float(row['amplitude']),
        float(row['significance']),
        float(row['ero_lcclass']),
        row['ero_date'],  # text, no conversion needed
        float(row['nh']),
        float(row['ero_gamma']),
        float(row['ero_gamma_err']),
        row['ero_cstat_dof'],  # text, no conversion
        float(row['ero_flux_mod']),
        float(row['ero_flux_mod_err'])
    )
    cursor.execute(insert_sql, values)

conn.commit()

In [7]:
cursor.execute("DROP TABLE IF EXISTS archival_xray;")

cursor.execute("""
CREATE TABLE archival_xray (
    id INTEGER PRIMARY KEY AUTOINCREMENT,  -- unique ID for each row
    ero_name TEXT NOT NULL,
    arch_flag REAL,
    arch_date TEXT,
    arch_flux REAL,
    arch_flux_err REAL,
    arch_instrument TEXT,
    FOREIGN KEY (ero_name) REFERENCES eroextra_master_erosita(ero_name)
);
""")


insert_sql = """
INSERT INTO archival_xray (
    ero_name, arch_flag, arch_date, arch_flux, arch_flux_err, arch_instrument
) VALUES (?, ?, ?, ?, ?, ?)
"""

for row in eroextra_data:
    ero_name = safe_field_access(row, "ero_name")
    
    arch_flag = parse_numeric_field(safe_field_access(row, "ARCH_FLAG"))
    arch_date = parse_numeric_field(safe_field_access(row, "ARCH_DATE"))
    arch_flux = parse_numeric_field(safe_field_access(row, "ARCH_FLUX"))
    arch_flux_err = parse_numeric_field(safe_field_access(row, "ARCH_FLUX_ERR"))
    arch_instrument = parse_instrument_field(safe_field_access(row, "ARCH_INSTRUMENT"))

    n_obs = max(len(arch_flag), len(arch_date), len(arch_flux), len(arch_flux_err), len(arch_instrument))

    def safe_get(lst, idx, default):
        return lst[idx] if idx < len(lst) else default

    for i in range(n_obs):
        values = (
            ero_name,
            safe_get(arch_flag, i, None),
            str(safe_get(arch_date, i, "")) if safe_get(arch_date, i, None) is not None else None,
            safe_get(arch_flux, i, None),
            safe_get(arch_flux_err, i, None),
            safe_get(arch_instrument, i, "")
        )
        cursor.execute(insert_sql, values)

conn.commit()


In [8]:
# Create the optical_follow_up table
cursor.execute("DROP TABLE IF EXISTS optical_follow_up;")
cursor.execute("""
CREATE TABLE optical_follow_up (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ero_name TEXT,
    ra REAL,
    dec REAL,
    redshift REAL,
    date TEXT,
    telescope TEXT,
    instrument TEXT,
    FOREIGN KEY (ero_name) REFERENCES eroextra_master_erosita(ero_name)
);
""")

insert_sql = """
INSERT INTO optical_follow_up 
    (ero_name, ra, dec, redshift, date, telescope, instrument) 
VALUES (?, ?, ?, ?, ?, ?, ?);
"""

# Insert all rows in a batch
cursor.executemany(insert_sql, optical_data)

In [9]:
# Create the optical_nway_counterparts table
cursor.execute("DROP TABLE IF EXISTS optical_nway_counterparts;")

cursor.execute("""
CREATE TABLE optical_nway_counterparts (
    ero_name TEXT,
    nway_bias_ls10_xray_proba REAL,
    nway_dist_bayesfactor REAL,
    nway_dist_post REAL,
    nway_p_single REAL,
    nway_p_any REAL,
    nway_p_i REAL,
    ls10_release INTEGER,
    ls10_brickid INTEGER,
    ls10_objid INTEGER,
    ls10_ra REAL,
    ls10_dec REAL,
    ls10_ra_ivar REAL,
    ls10_dec_ivar REAL,
    ls10_flux_g REAL,
    ls10_flux_r REAL,
    ls10_flux_i REAL,
    ls10_flux_z REAL,
    ls10_flux_w1 REAL,
    ls10_flux_w2 REAL,
    ls10_flux_ivar_g REAL,
    ls10_flux_ivar_r REAL,
    ls10_flux_ivar_i REAL,
    ls10_flux_ivar_z REAL,
    ls10_flux_ivar_w1 REAL,
    ls10_flux_ivar_w2 REAL,
    ls10_mw_transmission_g REAL,
    ls10_mw_transmission_r REAL,
    ls10_mw_transmission_i REAL,
    ls10_mw_transmission_z REAL,
    ls10_mw_transmission_w1 REAL,
    ls10_mw_transmission_w2 REAL,
    ls10_type TEXT,
    z_redshift REAL,
    z_type TEXT,
    z_reference TEXT,
    FOREIGN KEY (ero_name) REFERENCES eroextra_master_erosita(ero_name)
);
""")

# Insert template
insert_sql_nway = """
INSERT OR REPLACE INTO optical_nway_counterparts (
    ero_name,
    nway_bias_ls10_xray_proba, nway_dist_bayesfactor, nway_dist_post, nway_p_single, nway_p_any, nway_p_i,
    ls10_release, ls10_brickid, ls10_objid, ls10_ra, ls10_dec, ls10_ra_ivar, ls10_dec_ivar,
    ls10_flux_g, ls10_flux_r, ls10_flux_i, ls10_flux_z, ls10_flux_w1, ls10_flux_w2,
    ls10_flux_ivar_g, ls10_flux_ivar_r, ls10_flux_ivar_i, ls10_flux_ivar_z, ls10_flux_ivar_w1, ls10_flux_ivar_w2,
    ls10_mw_transmission_g, ls10_mw_transmission_r, ls10_mw_transmission_i, ls10_mw_transmission_z,
    ls10_mw_transmission_w1, ls10_mw_transmission_w2,
    ls10_type, z_redshift, z_type, z_reference
) VALUES (
    ?, ?, ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?, ?,
    ?, ?, ?, ?, ?, ?,
    ?, ?, ?, ?,
    ?, ?,
    ?, ?, ?, ?
);
"""

# Populate the table
for row in eroextra_data:   # assuming your loaded table is called nway_data
    values = (
        row['ero_name'],
        float(row['nway_bias_ls10_xray_proba']),
        float(row['nway_dist_bayesfactor']),
        float(row['nway_dist_post']),
        float(row['nway_p_single']),
        float(row['nway_p_any']),
        float(row['nway_p_i']),
        int(row['ls10_release']),
        int(row['ls10_brickid']),
        int(row['ls10_objid']),
        float(row['ls10_ra']),
        float(row['ls10_dec']),
        float(row['ls10_ra_ivar']),
        float(row['ls10_dec_ivar']),
        float(row['ls10_flux_g']),
        float(row['ls10_flux_r']),
        float(row['ls10_flux_i']),
        float(row['ls10_flux_z']),
        float(row['ls10_flux_w1']),
        float(row['ls10_flux_w2']),
        float(row['ls10_flux_ivar_g']),
        float(row['ls10_flux_ivar_r']),
        float(row['ls10_flux_ivar_i']),
        float(row['ls10_flux_ivar_z']),
        float(row['ls10_flux_ivar_w1']),
        float(row['ls10_flux_ivar_w2']),
        float(row['ls10_mw_transmission_g']),
        float(row['ls10_mw_transmission_r']),
        float(row['ls10_mw_transmission_i']),
        float(row['ls10_mw_transmission_z']),
        float(row['ls10_mw_transmission_w1']),
        float(row['ls10_mw_transmission_w2']),
        row['ls10_type'],
        float(row['z_redshift']),
        row['z_type'],
        row['z_reference']
    )
    cursor.execute(insert_sql_nway, values)

conn.commit()


In [10]:
# Create the simbad_matches table
cursor.execute("DROP TABLE IF EXISTS simbad_matches;")

cursor.execute("""
CREATE TABLE simbad_matches (
    ls10_ra REAL,
    ls10_dec REAL,
    simbad_name TEXT,
    simbad_ra REAL,
    simbad_dec REAL,
    simbad_type TEXT,
    FOREIGN KEY (ls10_ra, ls10_dec) 
        REFERENCES optical_nway_counterparts(ls10_ra, ls10_dec)
);
""")

# Insert template
insert_sql_simbad = """
INSERT OR REPLACE INTO simbad_matches (
    ls10_ra, ls10_dec,
    simbad_name, simbad_ra, simbad_dec, simbad_type
) VALUES (
    ?, ?, ?, ?, ?, ?
);
"""

# Populate only if simbad_name is not empty
for row in eroextra_data:   # same FITS table as before
    simbad_name = row['simbad_name']
    if simbad_name not in (None, '', ' '):  # skip empty matches
        values = (
            float(row['ls10_ra']),
            float(row['ls10_dec']),
            simbad_name,
            float(row['simbad_ra']) if row['simbad_ra'] is not None else None,
            float(row['simbad_dec']) if row['simbad_dec'] is not None else None,
            row['simbad_type']
        )
        cursor.execute(insert_sql_simbad, values)

conn.commit()


In [11]:
# Create tns_matches table
cursor.execute("DROP TABLE IF EXISTS tns_matches;")

cursor.execute("""
CREATE TABLE tns_matches (
    ls10_ra REAL,
    ls10_dec REAL,

    tns_name TEXT,
    tns_ra REAL,
    tns_dec REAL,
    tns_date REAL,

    FOREIGN KEY (ls10_ra, ls10_dec)
        REFERENCES optical_nway_counterparts(ls10_ra, ls10_dec)
);
""")

# Insert into tns_matches only if tns_name exists
insert_sql_tns = """
INSERT OR REPLACE INTO tns_matches (
    ls10_ra, ls10_dec,
    tns_name, tns_ra, tns_dec, tns_date
) VALUES (?, ?, ?, ?, ?, ?);
"""

for row in eroextra_data:  # your FITS rows
    if row['tns_name'] not in (None, '', ' '):
        values = (
            float(row['ls10_ra']),
            float(row['ls10_dec']),
            row['tns_name'],
            float(row['tns_ra']) if row['tns_ra'] is not None else None,
            float(row['tns_dec']) if row['tns_dec'] is not None else None,
            float(row['tns_date']) if row['tns_date'] is not None else None
        )
        cursor.execute(insert_sql_tns, values)

        

conn.commit()


In [12]:
# Create radio_matches table
cursor.execute("DROP TABLE IF EXISTS radio_matches;")

cursor.execute("""
CREATE TABLE radio_matches (
    ls10_ra REAL,
    ls10_dec REAL,

    -- VLASS
    vlass_name TEXT,
    vlass_ra REAL,
    vlass_dec REAL,
    vlass_flux REAL,

    -- RACS
    racs_name TEXT,
    racs_ra REAL,
    racs_dec REAL,
    racs_flux REAL,

    FOREIGN KEY (ls10_ra, ls10_dec)
        REFERENCES optical_nway_counterparts(ls10_ra, ls10_dec)
);
""")

insert_sql_radio = """
INSERT OR REPLACE INTO radio_matches (
    ls10_ra, ls10_dec,
    vlass_name, vlass_ra, vlass_dec, vlass_flux,
    racs_name, racs_ra, racs_dec, racs_flux
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

# Insert into radio_matches only if vlass_name or racs_name exists
for row in eroextra_data:
    vlass_exists = row['vlass_name'] not in (None, '', ' ')
    racs_exists  = row['racs_name'] not in (None, '', ' ')

    if vlass_exists or racs_exists:  # allow either or both
        values = (
            float(row['ls10_ra']),
            float(row['ls10_dec']),

            row['vlass_name'] if vlass_exists else None,
            float(row['vlass_ra']) if vlass_exists and row['vlass_ra'] is not None else None,
            float(row['vlass_dec']) if vlass_exists and row['vlass_dec'] is not None else None,
            float(row['vlass_flux']) if vlass_exists and row['vlass_flux'] is not None else None,

            row['racs_name'] if racs_exists else None,
            float(row['racs_ra']) if racs_exists and row['racs_ra'] is not None else None,
            float(row['racs_dec']) if racs_exists and row['racs_dec'] is not None else None,
            float(row['racs_flux']) if racs_exists and row['racs_flux'] is not None else None
        )
        cursor.execute(insert_sql_radio, values)

conn.commit()


In [13]:
#commit and close the connection
conn.commit()
conn.close()

In [14]:
conn = sqlite3.connect("eroextra.db")
cursor = conn.cursor()

# Show all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

conn.close()

[('sqlite_sequence',), ('eroextra_master_erosita',), ('archival_xray',), ('optical_follow_up',), ('optical_nway_counterparts',), ('simbad_matches',), ('tns_matches',), ('radio_matches',)]
